<a href="https://colab.research.google.com/github/raulbs7/Machine-Learning-Techniques-Project/blob/master/NLP_Supervised_Project/2_Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2. VECTORIZATION

## 2.1 Imports

First of all, as always, we do the imports for the project.

In [2]:
from google.colab import files
from google.colab import drive
import pandas as pd
import nltk
import io
import re
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import word_tokenize

nltk.download("popular")
nltk.download('vader_lexicon') #sentiment analysis
nltk.download('twython') #twitter

from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
drive.mount('/drive')

Mounted at /drive


## 2.2 Importing dataset

The dataset that it is going to take for this phase is called **processed_tweets.csv**

In [4]:
def upload_dataframes (index_fields):
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    df = pd.read_csv(io.StringIO(uploaded[fn].decode('utf-8')), index_col = index_fields)
    return df

In [5]:
tweets = upload_dataframes([])

Saving processed_tweets.csv to processed_tweets.csv
User uploaded file "processed_tweets.csv" with length 1558482 bytes


In [6]:
print(tweets.shape)
tweets.head()

(24783, 7)


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,rt woman complain cleaning house man always ta...
1,1,3,0,3,0,1,rt boy dat coldtyga dwn bad cuffin dat hoe 1st...
2,2,3,0,3,0,1,rt dawg rt ever fuck bitch start cry confused ...
3,3,3,0,2,1,1,rt look like tranny
4,4,6,0,6,0,1,rt shit hear might true might faker bitch told...


## 2.3 TFIDF

The first configuration that is going to be extracted form the tweets is TFIDF, that it is similar to the CountVectorizer, but the principal difference is that TFIDF can express the relevance of each feature. The criteria is going to be to remove the features that doesn't appear in 4 tweets at least.

In [7]:
def tfidf_vectorizer(matrix):
  vectorizer = TfidfVectorizer(min_df=4, lowercase=False, stop_words='english')
  return vectorizer.fit_transform(matrix)

In [8]:
tweets_tfidf = tfidf_vectorizer(tweets['tweet'])
tweets_tfidf

<24783x4522 sparse matrix of type '<class 'numpy.float64'>'
	with 146734 stored elements in Compressed Sparse Row format>

## 2.3 TFIDF with N-grams

This is similar to the regular TFIDF configuration. However, this configuration is to obtain some n-grams. The n-grams are sets of words that could appear nearly. The criteria for choosing n-grams is going to be the same as the first configuration.

In [9]:
def tfidf_ngram_vectorizer(matrix):
  vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=4, lowercase=False, stop_words='english')
  return vectorizer.fit_transform(matrix)

In [10]:
tweets_ngrams = tfidf_ngram_vectorizer(tweets['tweet'])
tweets_ngrams

<24783x8048 sparse matrix of type '<class 'numpy.float64'>'
	with 183087 stored elements in Compressed Sparse Row format>

## 2.3 TFIDF with N-grams and POS-tagging

This consists in tokenize the tweets to words, and obtain the pos-tags that belongs to each word. This pos-tags are going to be vectorized the same as the other vectorizations, but instead of having tweets and words, we are going to have pos tags.


In [11]:
def tokenize_pos(raw):
  tokens = word_tokenize(raw)
  tags = nltk.pos_tag(tokens)
  raw_tags = " ".join(tag for (word, tag) in tags)
  return raw_tags

In [12]:
tokens_tweet = tweets['tweet'].apply(tokenize_pos)
tokens_tweet.head()

0          NNS NN VBP NN NN NN RB VBP NN
1       NN NN NN NN NN JJ NN NN NN CD NN
2        NN NN NN RB VBD JJ NN NN VBD NN
3                            JJ NN IN NN
4    NN VBD JJ MD JJ MD VB NN VBD CD NNS
Name: tweet, dtype: object

In [13]:
tweets_ngrams_pos = tfidf_ngram_vectorizer(tokens_tweet)
tweets_ngrams_pos

<24783x381 sparse matrix of type '<class 'numpy.float64'>'
	with 221389 stored elements in Compressed Sparse Row format>

## 2.4 TFIDF with N-grams, POS-tagging and other features.

### Number of RT's

Another features is going to be the number of RT's words that a tweet had.

In [14]:
def num_rt(raw):
  rt = 0
  words = raw.split()
  for word in words:
    if word == 'rt':
      rt += 1
  return rt

In [15]:
tweets_rt = tweets['tweet'].apply(num_rt)
tweets_rt.head()

0    1
1    1
2    2
3    1
4    1
Name: tweet, dtype: int64

In [16]:
tweets_rt = np.reshape(tweets_rt.to_list(), (len(tweets_rt), 1))
tweets_rt

array([[1],
       [1],
       [2],
       ...,
       [0],
       [0],
       [0]])

In [17]:
tweets_rt = sp.csr_matrix(tweets_rt)
tweets_rt

<24783x1 sparse matrix of type '<class 'numpy.longlong'>'
	with 7130 stored elements in Compressed Sparse Row format>

###Sentiment Analysis

Also, the tweets are going to analyze and treated by a sentiment analyzer, that is going to give us 4 different feautures representing different sentiments about that tweet.

In [18]:
def sentiment_analysis(raw):
  sentiment_analyzer  = SentimentIntensityAnalyzer() 
  sentiment = sentiment_analyzer.polarity_scores(raw)
  return [value for key, value in sentiment.items()]

In [19]:
tweets_sentiment = tweets['tweet'].apply(sentiment_analysis)
tweets_sentiment.head()

0     [0.238, 0.762, 0.0, -0.3612]
1     [0.259, 0.741, 0.0, -0.5423]
2      [0.765, 0.235, 0.0, -0.946]
3      [0.0, 0.545, 0.455, 0.3612]
4    [0.43, 0.407, 0.163, -0.6808]
Name: tweet, dtype: object

In [20]:
tweets_sentiment = sp.csr_matrix(tweets_sentiment.to_list())
tweets_sentiment

<24783x4 sparse matrix of type '<class 'numpy.float64'>'
	with 72757 stored elements in Compressed Sparse Row format>

###Hatred N-gram dictionary

Lastly, with the hatred n-gram dictionary, it is going to be compared the words in this dictionary withe the words of the tweets. If a word of this dictionary appears in a tweet, a weight is going to be associated to it.

In [21]:
hatred_dictionary = upload_dataframes([])

Saving refined_ngram_dict.csv to refined_ngram_dict.csv
User uploaded file "refined_ngram_dict.csv" with length 3178 bytes


In [22]:
hatred_dictionary.head()

,ngram,prophate
0,allah akbar,0.870
1,blacks,0.583
2,chink,0.467
3,chinks,0.542
4,dykes,0.602


In [23]:
def hatred_dict_analysis(raw):
  weights = []
  for i in hatred_dictionary.index:
    if re.search(hatred_dictionary['ngram'][i], raw) is None:
      weights.append(0.0)
    else:
      weights.append(hatred_dictionary['prophate'][i])
  return weights

In [24]:
tweets_hatred_dict = tweets['tweet'].apply(hatred_dict_analysis)
tweets_hatred_dict.head()

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: tweet, dtype: object

In [25]:
tweets_hatred_dict = sp.csr_matrix(tweets_hatred_dict.to_list())
tweets_hatred_dict

<24783x178 sparse matrix of type '<class 'numpy.float64'>'
	with 1367 stored elements in Compressed Sparse Row format>

##2.5 Unify all configurations

The configurations used are going to be the next:


*   TFIDF with n-grams
*   TFIDF with n-grams of POS tagging
*   Number of RT's
*   Sentiments of tweets
*   Hatred words

TFIDF without n-grams because it is implictly incorporated in the vectorization with n-grams yet.



In [26]:
configurations = sp.hstack((tweets_ngrams, tweets_ngrams_pos,
                            tweets_rt, tweets_sentiment, tweets_hatred_dict), format='csr')
configurations

<24783x8612 sparse matrix of type '<class 'numpy.float64'>'
	with 485730 stored elements in Compressed Sparse Row format>

After concatenate all the configuration matrices, one thing needed to do is to remove columns that have zeros in all the entries.

In [34]:
correct_columns = np.where(configurations.sum(axis=0)!=0)[1]
configurations = configurations[:,correct_columns]
configurations

<24783x8494 sparse matrix of type '<class 'numpy.float64'>'
	with 485730 stored elements in Compressed Sparse Row format>

We convert that matrix into a DataFrame.

In [35]:
configurations_df = pd.DataFrame(configurations.toarray())
configurations_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,8454,8455,8456,8457,8458,8459,8460,8461,8462,8463,8464,8465,8466,8467,8468,8469,8470,8471,8472,8473,8474,8475,8476,8477,8478,8479,8480,8481,8482,8483,8484,8485,8486,8487,8488,8489,8490,8491,8492,8493
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.338969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


On the last hand, it is necessary to convert the the dataframe into a CSV file.

In [36]:
tweets.to_csv('/drive/My Drive/vectorized_tweets.csv')